In [1]:
import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import sklearn
import pandas as pd
import os
import sys
import time
import tensorflow as tf
from tensorflow import keras

print(tf.__version__)
print(sys.version_info)
for module in mpl,np,pd,sklearn,tf,keras:
    print(module.__name__,module.__version__)

2.0.0
sys.version_info(major=3, minor=7, micro=4, releaselevel='final', serial=0)
matplotlib 3.1.1
numpy 1.16.5
pandas 0.25.2
sklearn 0.21.3
tensorflow 2.0.0
tensorflow_core.keras 2.2.4-tf


In [2]:
from sklearn.datasets import fetch_california_housing

# 房价预测
housing = fetch_california_housing()


In [3]:
# 划分样本
from sklearn.model_selection import train_test_split

x_train_all,x_test,y_train_all,y_test = train_test_split(housing.data,housing.target,random_state=7)
x_train,x_valid,y_train,y_valid = train_test_split(x_train_all,y_train_all,random_state=11)

print(x_train.shape,y_train.shape)
print(x_valid.shape,y_valid.shape)
print(x_test.shape,y_test.shape)


(11610, 8) (11610,)
(3870, 8) (3870,)
(5160, 8) (5160,)


In [4]:
# 归一化
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
x_train_scaled = scaler.fit_transform(x_train)
x_valid_scaled = scaler.transform(x_valid)
x_test_scaled = scaler.transform(x_test)

In [5]:
output_dir = 'generate_csv'
if not os.path.exists(output_dir):
    os.mkdir(output_dir)
    
def save_to_csv(output_dir,data,name_prefix,header=None,n_parts=10):
    
    path_format = os.path.join(output_dir,'{}_{:02d}.csv')
    filenames = []
    
    for file_idx,row_indices in enumerate(np.array_split(np.arange(len(data)),n_parts)):
        part_csv = path_format.format(name_prefix,file_idx)
        filenames.append(part_csv)
        
        with open(part_csv,'wt',encoding='utf-8') as f:
            if header is not None:
                f.write(header + '\n')
            for row_index in row_indices:
                f.write(','.join([repr(col) for col in data[row_index]]))
                f.write('\n')
    
    return filenames
    

train_data = np.c_[x_train_scaled,y_train]
valid_data = np.c_[x_valid_scaled,y_valid]
test_data = np.c_[x_test_scaled,y_test]

header_cols = housing.feature_names + ['MiddianHouseValue']

header_str = ','.join(header_cols)

In [6]:
train_filenames = save_to_csv(output_dir,train_data,'train',header_str,n_parts=20)
valid_filenames = save_to_csv(output_dir,valid_data,'valid',header_str,n_parts=10)
test_filenames = save_to_csv(output_dir,test_data,'test',header_str,n_parts=10)

In [9]:
# 读取上面生成的文件
# 打印文件名
import pprint
print("train filenames: ")
pprint.pprint(train_filenames)
print("valid filenames: ")
pprint.pprint(valid_filenames)
print("test filenames: ")
pprint.pprint(test_filenames)

train filenames: 
['generate_csv\\train_00.csv',
 'generate_csv\\train_01.csv',
 'generate_csv\\train_02.csv',
 'generate_csv\\train_03.csv',
 'generate_csv\\train_04.csv',
 'generate_csv\\train_05.csv',
 'generate_csv\\train_06.csv',
 'generate_csv\\train_07.csv',
 'generate_csv\\train_08.csv',
 'generate_csv\\train_09.csv',
 'generate_csv\\train_10.csv',
 'generate_csv\\train_11.csv',
 'generate_csv\\train_12.csv',
 'generate_csv\\train_13.csv',
 'generate_csv\\train_14.csv',
 'generate_csv\\train_15.csv',
 'generate_csv\\train_16.csv',
 'generate_csv\\train_17.csv',
 'generate_csv\\train_18.csv',
 'generate_csv\\train_19.csv']
valid filenames: 
['generate_csv\\valid_00.csv',
 'generate_csv\\valid_01.csv',
 'generate_csv\\valid_02.csv',
 'generate_csv\\valid_03.csv',
 'generate_csv\\valid_04.csv',
 'generate_csv\\valid_05.csv',
 'generate_csv\\valid_06.csv',
 'generate_csv\\valid_07.csv',
 'generate_csv\\valid_08.csv',
 'generate_csv\\valid_09.csv']
test filenames: 
['generate_csv\\t

In [11]:
# 读取文件
# 1. filename -> dataset
# 2. read file -> dataset -> datasets -> megre
# 3. parse csv

filename_dataset = tf.data.Dataset.list_files(train_filenames)
for filename  in filename_dataset:
    print(filename)

tf.Tensor(b'generate_csv\\train_10.csv', shape=(), dtype=string)
tf.Tensor(b'generate_csv\\train_13.csv', shape=(), dtype=string)
tf.Tensor(b'generate_csv\\train_04.csv', shape=(), dtype=string)
tf.Tensor(b'generate_csv\\train_12.csv', shape=(), dtype=string)
tf.Tensor(b'generate_csv\\train_08.csv', shape=(), dtype=string)
tf.Tensor(b'generate_csv\\train_15.csv', shape=(), dtype=string)
tf.Tensor(b'generate_csv\\train_06.csv', shape=(), dtype=string)
tf.Tensor(b'generate_csv\\train_19.csv', shape=(), dtype=string)
tf.Tensor(b'generate_csv\\train_16.csv', shape=(), dtype=string)
tf.Tensor(b'generate_csv\\train_05.csv', shape=(), dtype=string)
tf.Tensor(b'generate_csv\\train_09.csv', shape=(), dtype=string)
tf.Tensor(b'generate_csv\\train_01.csv', shape=(), dtype=string)
tf.Tensor(b'generate_csv\\train_17.csv', shape=(), dtype=string)
tf.Tensor(b'generate_csv\\train_07.csv', shape=(), dtype=string)
tf.Tensor(b'generate_csv\\train_14.csv', shape=(), dtype=string)
tf.Tensor(b'generate_csv\

In [13]:
n_readers = 5 # 并行度
dataset = filename_dataset.interleave(
    lambda filename : tf.data.TextLineDataset(filename).skip(1), # skip(1) 省略一行
    cycle_length = n_readers
)

for line in dataset.take(15):
    print(line.numpy())

b'0.8115083791797953,-0.04823952235146133,0.5187339067174729,-0.029386394873127775,-0.034064024638222286,-0.05081594842905086,-0.7157356834231196,0.9162751241885168,2.147'
b'-1.0775077698160966,-0.44874070548966555,-0.5680568205591913,-0.14269262164909954,-0.09666677138213985,0.12326468238687088,-0.3144863716683942,-0.4818958888413162,0.978'
b'0.401276648075221,-0.9293421252555106,-0.05333050451405854,-0.1865945262276826,0.6545661895448709,0.026434465728210874,0.9312527706398824,-1.4406417263474771,2.512'
b'0.04326300977263167,-1.0895425985107923,-0.38878716774583305,-0.10789864528874438,-0.6818663605100649,-0.0723871014747467,-0.8883662012710817,0.8213992340186296,1.426'
b'-0.8219588176953616,1.874166156711919,0.18212349433218608,-0.03170019246279883,-0.6011178900722581,-0.14337494105109344,1.0852205298015787,-0.8613994495208361,1.054'
b'-0.6906143291679195,-0.1283397589791022,7.0201810347470595,5.624287386169439,-0.2663292879200034,-0.03662080416157129,-0.6457503383496215,1.205896262

In [14]:
# parse csv 字符串变成tensor
# tf.io.decode_csv(str,record_defaults,)

sample_str = '1,2,3,4,5'
record_defaults = [tf.constant(0,dtype=tf.int32)] * 5
parsed_fields = tf.io.decode_csv(sample_str,record_defaults)
print(parsed_fields)

[<tf.Tensor: id=136, shape=(), dtype=int32, numpy=1>, <tf.Tensor: id=137, shape=(), dtype=int32, numpy=2>, <tf.Tensor: id=138, shape=(), dtype=int32, numpy=3>, <tf.Tensor: id=139, shape=(), dtype=int32, numpy=4>, <tf.Tensor: id=140, shape=(), dtype=int32, numpy=5>]


In [15]:
sample_str = '1,2,3,4,5'
record_defaults = [tf.constant(0,dtype=tf.int32),
                  0,
                  np.nan,
                  'hello',
                  tf.constant([])]
parsed_fields = tf.io.decode_csv(sample_str,record_defaults)
print(parsed_fields)

[<tf.Tensor: id=147, shape=(), dtype=int32, numpy=1>, <tf.Tensor: id=148, shape=(), dtype=int32, numpy=2>, <tf.Tensor: id=149, shape=(), dtype=float32, numpy=3.0>, <tf.Tensor: id=150, shape=(), dtype=string, numpy=b'4'>, <tf.Tensor: id=151, shape=(), dtype=float32, numpy=5.0>]


In [16]:
try:
    parsed_fields = tf.io.decode_csv(',,,,',record_defaults)
except tf.errors.InvalidArgumentError as ex:
    print(ex)

Field 4 is required but missing in record 0! [Op:DecodeCSV]


In [17]:
try:
    parsed_fields = tf.io.decode_csv('1,2,3,4,5,6,7',record_defaults)
except tf.errors.InvalidArgumentError as ex:
    print(ex)

Expect 5 fields but have 7 in record 0 [Op:DecodeCSV]


In [24]:
# 解析dataset 中的一行

def parse_csv_line(line,n_fields=9):
    defs = [tf.constant(np.nan)] * n_fields
    parsed_fields = tf.io.decode_csv(line,record_defaults=defs)
    x = tf.stack(parsed_fields[0:-1])
    y = tf.stack(parsed_fields[-1:])
    
    return x,y
    
parse_csv_line(b'-0.060214068004363165,0.7527628439249472,0.0835940301935345,-0.06250122441959183,-0.03497131082291674,-0.026442380178345683,1.0712234607868782,-1.3707331756959855,1.651',
              n_fields=9)

(<tf.Tensor: id=279, shape=(8,), dtype=float32, numpy=
 array([-0.06021407,  0.75276285,  0.08359403, -0.06250122, -0.03497131,
        -0.02644238,  1.0712235 , -1.3707331 ], dtype=float32)>,
 <tf.Tensor: id=280, shape=(1,), dtype=float32, numpy=array([1.651], dtype=float32)>)

In [25]:
# 整体
# 1. filename -> dataset
# 2. read file -> dataset -> datasets -> megre
# 3. parse csv

def csv_reader_dataset(filenames,n_readers=5,batch_size=32,n_parse_threads=5,
                      shuffle_buffer_size=10000):
    
    # 1. filename -> dataset
    dataset = tf.data.Dataset.list_files(filenames)
    dataset = dataset.repeat() # 重复多少次
    # 文件名转换成文本内容
    dataset = dataset.interleave(
            lambda filename:tf.data.TextLineDataset(filename).skip(1),
            cycle_length = n_readers
    )
    dataset.shuffle(shuffle_buffer_size)
    # 解析
    dataset = dataset.map(parse_csv_line,
                         num_parallel_calls=n_parse_threads)
    
    dataset = dataset.batch(batch_size)
    return dataset

# 测试
train_set = csv_reader_dataset(train_filenames,batch_size=3)
for x_batch,y_batch in train_set.take(2):
    print("x:")
    pprint.pprint(x_batch)
    print("y:")
    pprint.pprint(y_batch)
    

x:
<tf.Tensor: id=366, shape=(3, 8), dtype=float32, numpy=
array([[-1.119975  , -1.3298433 ,  0.14190045,  0.4658137 , -0.10301778,
        -0.10744184, -0.7950524 ,  1.5304717 ],
       [-1.119975  , -1.3298433 ,  0.14190045,  0.4658137 , -0.10301778,
        -0.10744184, -0.7950524 ,  1.5304717 ],
       [-1.119975  , -1.3298433 ,  0.14190045,  0.4658137 , -0.10301778,
        -0.10744184, -0.7950524 ,  1.5304717 ]], dtype=float32)>
y:
<tf.Tensor: id=367, shape=(3, 1), dtype=float32, numpy=
array([[0.66],
       [0.66],
       [0.66]], dtype=float32)>
x:
<tf.Tensor: id=368, shape=(3, 8), dtype=float32, numpy=
array([[-1.119975  , -1.3298433 ,  0.14190045,  0.4658137 , -0.10301778,
        -0.10744184, -0.7950524 ,  1.5304717 ],
       [-1.119975  , -1.3298433 ,  0.14190045,  0.4658137 , -0.10301778,
        -0.10744184, -0.7950524 ,  1.5304717 ],
       [-0.9490939 ,  0.6726626 ,  0.28370556,  0.1065553 , -0.65464777,
        -0.06239493,  0.21273656,  0.0024705 ]], dtype=float32)>
y

In [26]:
batch_size = 32
train_sest = csv_reader_dataset(train_filenames,batch_size=batch_size)
valid_set = csv_reader_dataset(valid_filenames,batch_size=batch_size)
test_set = csv_reader_dataset(test_filenames,batch_size=batch_size)

In [27]:
# 搭建模型
model = keras.models.Sequential([
    keras.layers.Dense(30,activation='relu',input_shape=[8]),
    keras.layers.Dense(1),
    
])

# 打印model信息
model.summary()
# 编译
model.compile(loss='mean_squared_error',optimizer="sgd")
# 回调函数
callbacks = [keras.callbacks.EarlyStopping(patience=5,min_delta=1e-3)]

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 30)                270       
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 31        
Total params: 301
Trainable params: 301
Non-trainable params: 0
_________________________________________________________________


In [29]:
#训练
history = model.fit(train_set,validation_data=valid_set,
                    steps_per_epoch=11160 // batch_size,
                    validation_steps = 3870 // batch_size,
                    epochs=100,callbacks=callbacks)

Train for 348 steps, validate for 120 steps
Epoch 1/100
348/348 [==============================] - 1s 4ms/step - loss: 0.4989 - val_loss: 0.6649
Epoch 2/100
348/348 [==============================] - 1s 3ms/step - loss: 1.1161 - val_loss: 0.9822
Epoch 3/100
348/348 [==============================] - 1s 3ms/step - loss: 0.6039 - val_loss: 0.5120
Epoch 4/100
348/348 [==============================] - 1s 3ms/step - loss: 0.6262 - val_loss: 0.6162
Epoch 5/100
348/348 [==============================] - 1s 3ms/step - loss: 0.3653 - val_loss: 0.5409
Epoch 6/100
348/348 [==============================] - 1s 3ms/step - loss: 0.4788 - val_loss: 0.4687
Epoch 7/100
348/348 [==============================] - 1s 2ms/step - loss: 0.3830 - val_loss: 0.4780
Epoch 8/100
348/348 [==============================] - 1s 2ms/step - loss: 0.3570 - val_loss: 0.5167
Epoch 9/100
348/348 [==============================] - 1s 2ms/step - loss: 0.4858 - val_loss: 0.4498
Epoch 10/100
348/348 [=========================

In [30]:
model.evaluate(test_set,steps = 5160 // batch_size)

161/161 [==============================] - 1s 4ms/step - loss: 0.4706


0.47063196596244106